In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import requests
import datetime as dt
from requests.auth import HTTPBasicAuth
from kaggle_secrets import UserSecretsClient

APP_ID  = UserSecretsClient().get_secret("APP_ID") # App id of your account on nutritionix.com
API_KEY = UserSecretsClient().get_secret("API_KEY") # Api key of your free account
NEUTRITIONIX_URL = "https://trackapi.nutritionix.com/v2/natural/exercise"
SHEETY_URL = UserSecretsClient().get_secret("SHEETY_URL") # URL of sheety associated with google sheet
USER = UserSecretsClient().get_secret("USER") # sheety user name
PASS = UserSecretsClient().get_secret("PASS") # sheety password

GENDER = "male"
WEIGHT_KG = 75
HEIGHT_CM = 204
AGE = 33


headers = {
    "Content-Type": "application/json",
    "x-app-id" : APP_ID ,
    "x-app-key" : API_KEY
}

exercise_text = input("Tell us which exercise you did: ") # e.g I ran 2 miles, I swim 15 minutes, I read 2 hours etc

params = {
    "query" : exercise_text,
    "gender": GENDER,
    "weight_kg": WEIGHT_KG,
    "height_cm": HEIGHT_CM,
    "age": AGE
}

response = requests.post(url=NEUTRITIONIX_URL, json=params, headers=headers)
response.raise_for_status()
data = response.json()

list_data = data['exercises']

current_datetime = dt.datetime.now()
date = current_datetime.strftime("%d/%m/%Y")
time = current_datetime.strftime("%X")

basic = HTTPBasicAuth(USER, PASS)

for exercise in data['exercises']:
    sheety_params = {
        "workout" : {
            "date" : date,
            "time" : time,
            "exercise" : exercise["name"].title(),
            "duration" : float(exercise['duration_min']),
            "calories" : float(exercise['nf_calories'])
    }
}

sheety_response = requests.post(url=SHEETY_URL, json=sheety_params,  auth=basic)
print(sheety_response)




# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session